---
bibliography: bio.bib
csl: harvard-cite-them-right.csl
title: Group Name's Group Project
execute:
  echo: false
  freeze: true
format:
  html:
    code-copy: true
    code-link: true
    toc: true
    toc-title: On this page
    toc-depth: 2
    toc_float:
      collapsed: false
      smooth_scroll: true
  pdf:
    include-in-header:
      text: |
        \addtokomafont{disposition}{\rmfamily}
    mainfont: Spectral
    sansfont: Roboto Flex
    monofont: InputMonoCondensed
    papersize: a4
    geometry:
      - top=25mm
      - left=40mm
      - right=30mm
      - bottom=25mm
      - heightrounded
    toc: false
    number-sections: false
    colorlinks: true
    highlight-style: github
jupyter:
  jupytext:
    text_representation:
      extension: .qmd
      format_name: quarto
      format_version: '1.0'
      jupytext_version: 1.16.4
  kernelspec:
    display_name: Python (base)
    language: python
    name: base
---

In [ ]:
#| echo: false
import os
import pandas as pd

In [ ]:
#| echo: false
host = 'https://orca.casa.ucl.ac.uk'
path = '~jreades/data'
file = '20240614-London-listings.parquet'

if os.path.exists(file):
  df = pd.read_parquet(file)
else: 
  df = pd.read_parquet(f'{host}/{path}/{file}')
  df.to_parquet(file)

## 1. Who collected the InsideAirbnb data?

::: {.duedate}

( 2 points; Answer due Week 7 )

:::

An inline citation example: As discussed on @insideairbnb, there are many...

A parenthetical citation example: There are many ways to research Airbnb [see, for example, @insideairbnb]... 

## 2. Why did they collect the InsideAirbnb data?

::: {.duedate}

( 4 points; Answer due Week 7 )

:::

This way is also supposed to work (`{python} f"{df.shape[0]:,}" `) but I've found it less reliable.

## 3. How did they collect it?

::: {.duedate}

( 5 points; Answer due Week 8 )

:::

## 4. How does the method of collection (Q3) impact the completeness and/or accuracy of the InsideAirbnb data? How well does it represent the process it seeks to study, and what wider issues does this raise?

::: {.duedate}

( 11 points; Answer due Week 9 )

:::

## 5. What ethical considerations does the use of the InsideAirbnb data raise? 

::: {.duedate}

( 18 points; Answer due {{< var assess.group-date >}} )

:::

## 6. With reference to the InsideAirbnb data (*i.e.* using numbers, figures, maps, and descriptive statistics), what does an analysis of Hosts and the types of properties that they list suggest about the nature of Airbnb lettings in London? 

::: {.duedate}

( 15 points; Answer due {{< var assess.group-date >}} )

:::


In [ ]:
#| echo: false
#| output: false
#Reading airbnb data 2013.12
import pandas as pd  
import numpy as np  
from scipy.stats import pearsonr, pointbiserialr, chi2_contingency  
import matplotlib.pyplot as plt  
from matplotlib.font_manager import FontProperties  
import seaborn as sns  
from matplotlib.gridspec import GridSpec  
import numpy as np  
import os  

# loading data
file_path = 'data/listings_202312.csv' 
airbnb_data = pd.read_csv(file_path)  
 
print(airbnb_data.info()) 

In [ ]:
#| echo: false
# estimate booking days per year by reviews and minimum nights
airbnb_data = airbnb_data[airbnb_data['availability_365'] > 0] 
airbnb_data['estimated_nights_booked'] = airbnb_data['reviews_per_month'] * 12 * airbnb_data['minimum_nights'] * 2  

#Data cleaning
# Replace NaN with 0
airbnb_data['estimated_nights_booked'] = airbnb_data['estimated_nights_booked'].fillna(0)
# Convert the column to integers
airbnb_data['estimated_nights_booked'] = airbnb_data['estimated_nights_booked'].astype(int)

#airbnb_data.to_csv('data/processed_airbnb_data.csv', index=False)  

### The 90-Day Rule and Airbnb Commercialization  

Airbnb has significantly impacted London’s housing market by reducing long-term housing availability, increasing rents, and driving commercialization. In response, London implemented the 90-day rule, restricting entire-home short-term rentals to 90 days annually unless planning permission is obtained [@greaterlondonauthority]. However, enforcement relies on self-reporting, making violations difficult to monitor. This analysis uses InsideAirbnb data to examine spatial patterns, booking behavior, host characteristics, and property types to assess Airbnb’s commercialization and compliance risks.  


In [ ]:
#| echo: false
#| warning: false
# 1. Data Cleaning and Preparation  
# -------------------------------  
# Select relevant columns and drop rows with missing values  
data = airbnb_data[['host_id', 'room_type', 'availability_365', 'calculated_host_listings_count',  
                    'reviews_per_month', 'minimum_nights', 'estimated_nights_booked',  
                    'price', 'latitude', 'longitude']].dropna()  

# Filter data where availability_365 is greater than 0  
data = data[data['availability_365'] > 0]  

import geopandas as gpd  
# Load the borough map as a GeoDataFrame  
borough_map = gpd.read_file("data/statistical-gis-boundaries-london/ESRI/London_Borough_Excluding_MHW.shp")

# Convert the cleaned data into a GeoDataFrame  
gdf = gpd.GeoDataFrame(  
    data,   
    geometry=gpd.points_from_xy(data['longitude'], data['latitude']),  
    crs="EPSG:4326"  # WGS84 coordinate system  
)  
# Reproject the GeoDataFrame to EPSG:27700 (British National Grid)  
gdf = gdf.to_crs("EPSG:27700")  
# Ensure both GeoDataFrames use the same CRS  
borough_map = borough_map.to_crs(gdf.crs)  

In [ ]:
#| echo: false
#| warning: false
import matplotlib.pyplot as plt  
import seaborn as sns  
from matplotlib.gridspec import GridSpec  
import numpy as np  
import os  # For creating directories  

# -------------------------------  
# 1. Setup the Main Figure Layout  
# -------------------------------  
fig = plt.figure(figsize=(16, 10))  # Main figure size  
gs = GridSpec(1, 1, figure=fig)  # Single grid for the map  

# Subplot positions and sizes  
x_offset = 0.75  
y_offsets = [0.82, 0.62, 0.42, 0.22]  
subplot_width, subplot_height = 0.18, 0.12  

# -------------------------------  
# 2. Plot the Map of Airbnb Listings  
# -------------------------------  
ax_map = fig.add_subplot(gs[0, 0])  
borough_map.plot(ax=ax_map, color='lightblue', edgecolor='darkblue', alpha=0.44)  

# Define room type colors  
room_type_colors = {  
    'Entire home/apt': '#FF7F7F',  
    'Private room': '#77DD77',  
    'Shared room': '#FDFD96',  
    'Hotel room': '#FFA07A'  
}  

# Plot Airbnb listings by room type  
for room_type, color in room_type_colors.items():  
    gdf[gdf['room_type'] == room_type].plot(  
        ax=ax_map, color=color, markersize=2, label=room_type, alpha=0.7  
    )  

# Add legend  
legend = ax_map.legend(  
    title="Room Type", loc='upper left', fontsize=10, frameon=True, edgecolor='lightgray'  
)  
legend.get_title().set_fontsize(10)  
legend.get_title().set_fontweight('bold')  

# Add map title  
plt.title('Airbnb Listings in London', fontsize=13)  

# Remove axes and borders for a cleaner map  
ax_map.set_xticks([]), ax_map.set_yticks([])  
for spine in ax_map.spines.values():  
    spine.set_visible(False)  

# -------------------------------  
# 3. Subplot 1: Estimated Booking Days  
# -------------------------------  
ax1 = fig.add_axes([x_offset, y_offsets[0], subplot_width, subplot_height])  

# Define bins and labels for booking days  
bins = [0, 30, 60, 89.999, 120, 150, 180, 210, 240, 270, 300, 330, 356]  
labels = ['0-30', '30-60', '60-90', '90-120', '120-150', '150-180', '180-210',  
          '210-240', '240-270', '270-300', '300-330', '330-356', '356+']  

# Assign colors based on booking days  
data['booking_color'] = data['estimated_nights_booked'].apply(  
    lambda x: 'blue' if x <= 89 else 'red'  
)  

# Plot histogram  
sns.histplot(  
    data=data, x='estimated_nights_booked', hue='booking_color',  
    palette={'blue': '#6A9FB5', 'red': '#FF6F61'}, multiple='dodge',  
    edgecolor='black', linewidth=0.5, binwidth=15, ax=ax1  
)  
ax1.set_title('1. Estimated Booking Days', fontsize=10, fontweight='bold')  
ax1.set_xlabel('Booking Days (0-356)', fontsize=8)  
ax1.set_ylabel('Listings', fontsize=8)  
ax1.tick_params(axis='x', rotation=45, labelsize=7)  
ax1.tick_params(axis='y', labelsize=7)  
ax1.set_xlim(0, 356)  

# Add vertical line at 90  
ax1.axvline(x=90, color='black', linestyle='--', linewidth=1)  

# Set custom x-axis ticks and labels  
ax1.set_xticks(bins)  
ax1.set_xticklabels(labels, rotation=45, fontsize=6.5)  

# Add custom legend  
ax1.legend(  
    handles=[  
        plt.Line2D([0], [0], color='#6A9FB5', lw=4, label='< 90'),  
        plt.Line2D([0], [0], color='#FF6F61', lw=4, label='>=90')  
    ],  
    title="Booking Days", loc='upper right', fontsize=8, title_fontsize=8  
)  
ax1.spines['top'].set_visible(False)  
ax1.spines['right'].set_visible(False)  

# -------------------------------  
# 4. Subplot 2: Listings per Host  
# -------------------------------  
ax2 = fig.add_axes([x_offset, y_offsets[1], subplot_width, subplot_height])  

# Group listings per host and create a new column for "10+"  
listings_per_host = data.groupby('host_id').size()  
listings_per_host = listings_per_host.apply(lambda x: x if x <= 10 else 11)  

# Plot histogram  
sns.histplot(  
    listings_per_host, kde=False, color='lightblue',  
    edgecolor='black', linewidth=0.5, binwidth=0.8, ax=ax2  
)  
ax2.set_title('2. Listings per Host', fontsize=10, fontweight='bold')  
ax2.set_xlabel('Number of Listings', fontsize=8)  
ax2.set_ylabel('Hosts', fontsize=8)  
ax2.tick_params(axis='x', labelsize=7)  
ax2.tick_params(axis='y', labelsize=7)  
ax2.set_xticks(range(1, 12))  
ax2.set_xticklabels([str(i) for i in range(1, 11)] + ['10+'])  
ax2.set_xlim(0.5, 11.5)  
ax2.spines['top'].set_visible(False)  
ax2.spines['right'].set_visible(False)  

# -------------------------------  
# 5. Subplot 3: Room Type Count  
# -------------------------------  
ax3 = fig.add_axes([x_offset, y_offsets[2], subplot_width, subplot_height])  

# Plot bar chart  
room_type_count = data['room_type'].value_counts()  
room_type_count.index = ['Entire home', 'Private room', 'Shared room', 'Hotel room']  
sns.barplot(  
    x=room_type_count.index, y=room_type_count.values,  
    palette=['#FF7F7F', '#AEC6CF', '#FDFD96', '#FFA07A'],  
    edgecolor='black', linewidth=0.5, width=0.4, ax=ax3  
)  
ax3.set_title('3. Room Type Count', fontsize=10, fontweight='bold')  
ax3.set_xlabel('Room Type', fontsize=8)  
ax3.set_ylabel('Count', fontsize=8)  
ax3.tick_params(axis='x', labelsize=7)  
ax3.tick_params(axis='y', labelsize=7)  
ax3.spines['top'].set_visible(False)  
ax3.spines['right'].set_visible(False)  

# -------------------------------  
# 6. Subplot 4: Minimum Nights Distribution  
# -------------------------------  
ax4 = fig.add_axes([x_offset, y_offsets[3], subplot_width, subplot_height])  

# Plot histogram  
sns.histplot(  
    data['minimum_nights'], bins=1000, kde=False,  
    color='lightcoral', edgecolor='black', linewidth=0.5, ax=ax4  
)  
ax4.set_title('4. Minimum Nights Distribution', fontsize=10, fontweight='bold')  
ax4.set_xlabel('Minimum Nights', fontsize=8)  
ax4.set_ylabel('Listings', fontsize=8)  
ax4.tick_params(axis='x', labelsize=7)  
ax4.tick_params(axis='y', labelsize=7)  
ax4.set_xlim(0, 40)  
ax4.spines['top'].set_visible(False)  
ax4.spines['right'].set_visible(False)  

# -------------------------------  
# 7. Add a Table Header  
# -------------------------------  
fig.suptitle('Figure 1: Airbnb Listings Analysis in London', fontsize=18, fontweight='bold', y=0.95)  

# -------------------------------  
# 8. Save the Figure  
# -------------------------------  
os.makedirs('plots', exist_ok=True)  # Create 'plots' folder if it doesn't exist  
plt.savefig('plots/figure_1_Airbnb_Listings_in_London.png', dpi=600, bbox_inches='tight')  

# -------------------------------  
# 9. Display the Figure  
# -------------------------------  
plt.show()

### Key Findings  

1. **Central London is a Compliance Hotspot**:  
   Central London has the highest concentration of short-term rentals, especially entire homes in high-demand areas, making it a key area for potential violations of the 90-day rule (Figure 1, Map).  

2. **Frequent Breaches of the 90-Day Limit**:  
   Booking data, based on reviews, pricing, and minimum nights [@insideairbnb2023], shows that entire homes and hotel-like properties often exceed the 90-day limit. This poses significant challenges for enforcement, particularly in highly commercialized areas (Figure 1, Panel 1).  

To address the limitations of using only snapshot data, we merged datasets from December 2023 to September 2024 and removed duplicates to ensure greater accuracy：


In [ ]:
#| echo: false
import pandas as pd  
import matplotlib.pyplot as plt  
import seaborn as sns  
import os  
from matplotlib.colors import LinearSegmentedColormap  

# -------------------------------  
# 1. Load and Combine Data  
# -------------------------------  
# Define the folder and file names  
data_folder = "data"  
file_names = ["listings_20243.csv", "listings_20246.csv", "listings_20249.csv"]  

# Load and combine the datasets  
data_list = [pd.read_csv(os.path.join(data_folder, file)) for file in file_names]  
combined_data = pd.concat(data_list, ignore_index=True)  

# Combine with airbnb_data (2023 data)  
all_data = pd.concat([combined_data, airbnb_data], ignore_index=True)  

# -------------------------------  
# 2. Data Cleaning  
# -------------------------------  
# Filter out listings with no availability  
all_data = all_data[all_data['availability_365'] > 0]  

# Calculate estimated booking days  
all_data['estimated_nights_booked'] = (  
    all_data['reviews_per_month'] * 12 * all_data['minimum_nights'] * 2  
)  
all_data['estimated_nights_booked'] = all_data['estimated_nights_booked'].fillna(0).astype(int)  

# Remove duplicates  
all_data = all_data.drop_duplicates()  

# -------------------------------  
# 3. Create 'host_type' Column  
# -------------------------------  
# Ensure 'calculated_host_listings_count' exists  
if 'calculated_host_listings_count' not in all_data.columns:  
    raise KeyError("'calculated_host_listings_count' column is missing from the DataFrame.")  

# Create 'host_type' column  
all_data['host_type'] = all_data['calculated_host_listings_count'].apply(  
    lambda x: 'Single' if x == 1 else 'Multi'  
)  

# -------------------------------  
# 4. Calculate Metrics  
# -------------------------------  
# Host type proportions  
host_proportions = all_data['host_type'].value_counts(normalize=True) * 100  
host_proportions = host_proportions.round(2)  

# Average booking days by host type  
host_avg_booking_days = all_data.groupby('host_type')['estimated_nights_booked'].mean().round(2)  

# Room type proportions  
room_type_proportions = all_data['room_type'].value_counts(normalize=True) * 100  
room_type_proportions = room_type_proportions.round(2)  

# Average booking days by room type  
room_type_avg_days = all_data.groupby('room_type')['estimated_nights_booked'].mean().round(2)  

# Average price and minimum nights by host type and room type  
avg_price_heatmap = all_data.pivot_table(index='host_type', columns='room_type', values='price', aggfunc='mean').round(2)  
min_nights_heatmap = all_data.pivot_table(index='host_type', columns='room_type', values='minimum_nights', aggfunc='mean').round(2)  

In [ ]:
#| echo: false
# 5. Visualizations  
# -------------------------------  
fig = plt.figure(figsize=(16, 14))  

# Main title  
fig.suptitle('Figure 2: Analysis of Airbnb Listings by Host and Room Types', fontsize=16, fontweight='bold', y=0.95)  

# First Row: Tables  
# Left: Host Type Table  
ax1 = fig.add_axes([0.1, 0.7, 0.35, 0.15])  
ax1.axis('off')  
host_table_data = pd.DataFrame({  
    'Host Type': host_proportions.index,  
    'Proportion (%)': host_proportions.values,  
    'Avg Booking Days': host_avg_booking_days.values  
})  
host_table = ax1.table(  
    cellText=host_table_data.values,  
    colLabels=host_table_data.columns,  
    cellLoc='center',  
    loc='center',  
    bbox=[0, 0, 1, 1]  
)  
host_table.auto_set_font_size(False)  
host_table.set_fontsize(12)  
for key, cell in host_table.get_celld().items():  
    if key[0] == 0:  # Header row  
        cell.set_text_props(weight='bold')  
        cell.set_height(0.1)  
    else:  
        cell.set_height(0.15)  

# Add left-side title  
fig.text(0.15, 0.87, 'Host Type Distribution and Booking Trends', fontsize=13, fontweight='bold')  

# Right: Room Type Table  
ax2 = fig.add_axes([0.55, 0.7, 0.35, 0.15])  
ax2.axis('off')  
room_table_data = pd.DataFrame({  
    'Room Type': room_type_proportions.index,  
    'Proportion (%)': room_type_proportions.values,  
    'Avg Booking Days': room_type_avg_days.values  
})  
room_table = ax2.table(  
    cellText=room_table_data.values,  
    colLabels=room_table_data.columns,  
    cellLoc='center',  
    loc='center',  
    bbox=[0, 0, 1, 1]  
)  
room_table.auto_set_font_size(False)  
room_table.set_fontsize(12)  
for key, cell in room_table.get_celld().items():  
    if key[0] == 0:  # Header row  
        cell.set_text_props(weight='bold')  
        cell.set_height(0.1)  
    else:  
        cell.set_height(0.15)  

# Add right-side title  
fig.text(0.6, 0.87, 'Room Type Distribution and Booking Trends', fontsize=13, fontweight='bold')  

# Second Row: Bar Charts  
# Left: Host Type Bar Chart  
ax3 = fig.add_axes([0.1, 0.45, 0.35, 0.2])  
host_avg_booking_days.plot(kind='bar', color=sns.color_palette("Pastel1")[:2], edgecolor='black', ax=ax3)  
ax3.set_title('Average Estimated Booking Days by Host Type', fontsize=10, fontweight='bold')  
ax3.set_xlabel('Host Type')  
ax3.set_ylabel('Average Booking Days')  

# Right: Room Type Bar Chart  
ax4 = fig.add_axes([0.55, 0.45, 0.35, 0.2])  
room_type_colors = sns.color_palette("Pastel1")[:len(room_type_avg_days)]  
room_type_avg_days.plot(kind='bar', color=room_type_colors, edgecolor='black', ax=ax4)  
ax4.set_title('Average Estimated Booking Days by Room Type', fontsize=10, fontweight='bold')  
ax4.set_xlabel('Room Type')  
ax4.set_ylabel('Average Booking Days')  

# Add dividing line  
fig.add_axes([0.49, 0.4, 0.01, 0.5]).plot([0, 0], [0, 1], linestyle='--', color='black', linewidth=1)  
plt.gca().axis('off')  

# Third Row: Heatmaps  
# Left: Average Price Heatmap  
ax5 = fig.add_axes([0.1, 0.1, 0.4, 0.23])  
sns.heatmap(avg_price_heatmap, annot=True, fmt=".2f", cmap="Blues", cbar=True, ax=ax5)  
ax5.set_title('Average Price by Host and Room Types', fontsize=10, fontweight='bold')  

# Right: Minimum Nights Heatmap  
slightly_higher_saturation_reds = LinearSegmentedColormap.from_list(  
    "slightly_higher_saturation_reds", ["#fff5f5", "#fcd9d9", "#f8a8a8", "#f47474"]  
)  
ax6 = fig.add_axes([0.55, 0.1, 0.4, 0.22])  
sns.heatmap(min_nights_heatmap, annot=True, fmt=".2f", cmap=slightly_higher_saturation_reds, cbar=True, ax=ax6)  
ax6.set_title('Minimum Nights by Host and Room Types', fontsize=10, fontweight='bold')  

# Save and display  
plt.savefig('plots/figure_2_Airbnb_Listings_Analysis.png', dpi=600, bbox_inches='tight')  
plt.show()

3. **Multi-Listing Hosts Dominate the Market**:  
   Multi-listing hosts account for 62.38% of all Airbnb listings and have longer average booking durations (75.17 days) than single-listing hosts (66.61 days). They mainly manage entire homes and hotel-like properties, which are more commercialized with higher prices and shorter stays (Figure 2, Top Left and heatmap).  

4. **Entire Homes are the Highest Risk**:  
   Entire homes dominate the market (66.97%) and have the longest booking durations (78.38 days), making them most likely to break the 90-day limit. Hotel-like properties, while smaller in volume, show similar risks due to their short minimum stays (2.13 nights) and high commercial focus (Figure 2, Top Right and heatmap).  





### Summary and Next Steps  

**Spatial Insights**: Central London displays a high intensity of short-term rentals and frequent entire-home use, making it a key hotspot for potential 90-day rule violations. Further investigation is needed to establish enforcement priorities and evaluate the impact of short-term rentals on local housing.  

**Policy Violation Factors**: Multi-listing hosts and commercialized property types, such as entire homes and hotel-like properties, are strongly linked to increased booking durations and greater non-compliance risks. Factors like rental type, pricing, and location require further analysis—such as regression models—to better target enforcement and guide regulatory measures.  

InsideAirbnb data highlights London’s Airbnb market as highly commercialized and dominated by multi-listing hosts, with entire-home rentals posing significant risks of non-compliance with the 90-day rule. Effective enforcement is crucial to address these challenges.  

---  

## 7. Drawing on your previous answers, and supporting your response with evidence (*e.g.* figures, maps, EDA/ESDA, and simple statistical analysis/models drawing on experience from, e.g., CASA0007), how *could* the InsideAirbnb data set be used to inform the regulation of Short-Term Lets (STL) in London? 

::: {.duedate}

( 45 points; Answer due {{< var assess.group-date >}} )

:::

To provide evidence on violation hotspots, community impacts, and the drivers of policy violations, the next steps are:  

1. **Identifying Violation Hotspots and Impacts:**  
   Spatial clustering will identify areas with high concentrations of 90-day rule violations. These hotspots will be analyzed for impacts on local communities, focusing on rising rents, reduced housing availability, and displacement of long-term residents—issues tied to Airbnb-driven gentrification [@smith2006].  

2. **Analyzing Drivers of Violations:**  
   Regression analysis helps identify key factors driving these violations (e.g. property type, host behavior, pricie and location...)

This approach will highlight the need for effective regulation, clarify enforcement priorities, and enhance the focus and efficiency of enforcement efforts.


In [ ]:
#| echo: false
#| output: false
import warnings
warnings.filterwarnings('ignore')

import pandas as pd  
import numpy as np  
from scipy.stats import pearsonr, pointbiserialr, chi2_contingency  
import matplotlib.pyplot as plt  
from matplotlib.font_manager import FontProperties  

# Read the data  
file_path = 'data/listings.csv'    #which is the path in this repository  
airbnb_data = pd.read_csv(file_path)

# Calculate the estimation of nights booked for each listing
airbnb_data = airbnb_data[airbnb_data['availability_365'] > 0] 
airbnb_data['estimated_nights_booked'] = airbnb_data['reviews_per_month'] * 12 * airbnb_data['minimum_nights'] * 2 

# Data cleaning: assign the estimated nights booked to each borough
# Replace NaN with 0
airbnb_data['estimated_nights_booked'] = airbnb_data['estimated_nights_booked'].fillna(0)

# Convert the column to integers
airbnb_data['estimated_nights_booked'] = airbnb_data['estimated_nights_booked'].astype(int)

#Count the number of listings in each borough using 'neighbourhood' column
borough_counts = airbnb_data['neighbourhood'].value_counts()

# Filter the DataFrame to include only rows where estimated_nights_booked is greater than 90
filtered_data = airbnb_data[airbnb_data['estimated_nights_booked'] > 90]

#Count the number of listings with estimation of nights booked larger than 90 days in each borough
borough_counts_90 = filtered_data['neighbourhood'].value_counts()

# Merge the two series into a DataFrame
combined_data = pd.concat([borough_counts, borough_counts_90], axis=1, keys=['Total_listings', 'More_than_90'])

# Calculate the ratio of listings with more than 90 booked nights per total listings
combined_data['Ratio_of_more_than_90'] = combined_data['More_than_90'] / combined_data['Total_listings']

# Fill any NaN values that might occur if there are boroughs with no listings > 90 nights
combined_data['Ratio_of_more_than_90'] = combined_data['Ratio_of_more_than_90'].fillna(0)

# Data formatting and round to four decimal places
combined_data['Ratio_of_more_than_90'] = combined_data['Ratio_of_more_than_90'].apply(lambda x: round(x, 4))

# Rename the index label to 'Borough_name'
combined_data.index.rename('Borough_name', inplace=True)

# Load the borough codes
borough_code_file_path = 'data/borough_name_code.csv'
borough_codes = pd.read_csv(borough_code_file_path)

# Reset index in combined_data to turn the index into a regular column
combined_data.reset_index(inplace=True)
borough_codes.reset_index(inplace=True)

#Combine the ratio data and borough name with borough code by borough name
combined_data = pd.merge(combined_data, borough_codes[['Borough_name', 'Borough_code']], on='Borough_name', how='left')

# Set 'Borough_name' back as the index
combined_data.set_index('Borough_name', inplace=True)

# Save the updated DataFrame
combined_data.to_csv('data/borough_listings_ratio.csv', index=True)

In [ ]:
#| echo: false
#| output: false
# Moran analysis
import geopandas as gpd
import libpysal
from esda.moran import Moran, Moran_Local
import matplotlib.pyplot as plt
from libpysal.weights import Queen, KNN
import seaborn as sns
import os

# Load data
ratio = pd.read_csv("data/borough_listings_ratio.csv")
borough = gpd.read_file("data/statistical-gis-boundaries-london/ESRI/London_Borough_Excluding_MHW.shp")

# merge
borough_ratio = borough.merge(ratio, left_on="GSS_CODE", right_on="Borough_code")

# Calculate neighbors using Queen contiguity
weights = Queen.from_dataframe(borough_ratio)
weights.transform = 'r'  # Row-standardize the weights

os.makedirs('plots/raw', exist_ok=True)

# Global Moran's I
y = borough_ratio['Ratio_of_more_than_90']
moran = Moran(y, weights)
print(f"Global Moran's I: {moran.I:.3f}")
print(f"P-value: {moran.p_sim:.3f}")

# Moran Plot
def moran_plot(y, weights):
    lag = weights.sparse.dot(y)
    slope, intercept = np.polyfit(y, lag, 1)
    
    plt.figure(figsize=(10, 5))
    plt.scatter(y, lag)
    plt.plot(y, slope * y + intercept, 'r')
    plt.xlabel('Ratio of more than 90')
    plt.ylabel('Spatially Lagged Ratio')
    plt.title("Moran Plot of rule breaking Airbnbs")
    plt.axvline(y.mean(), color='gray', linestyle='--')
    plt.axhline(lag.mean(), color='gray', linestyle='--')
    plt.savefig('plots/raw/Moran_rule_breaking.png') 
    plt.show()

moran_plot(y, weights)

In [ ]:
#| echo: false
#| output: false
# Local Moran's I
local_moran = Moran_Local(y, weights)
borough_ratio['Ii'] = local_moran.Is
borough_ratio['p_value'] = local_moran.p_sim

# Plot Local Moran's I
fig, ax = plt.subplots(figsize=(12, 8))
borough_ratio.plot(column='Ii', legend=True, ax=ax)
plt.title("Local Moran's I Statistics")
plt.axis('off')
plt.show()

# LISA Cluster Map
sig = 0.1
labels = ['Not Significant', 'Low-Low', 'Low-High', 'High-Low', 'High-High']
colors = ['white', 'blue', 'lightblue', 'pink', 'red']

# Standardize the variable of interest
y_std = (y - y.mean()) / y.std()
lag_std = weights.sparse.dot(y_std)

# Create significance masks
sig_mask = local_moran.p_sim < sig

# Create cluster categories
borough_ratio['quadrant'] = np.zeros(len(y))
borough_ratio.loc[sig_mask, 'quadrant'] = np.where(y_std < 0,
    np.where(lag_std < 0, 1, 2),
    np.where(lag_std < 0, 3, 4))[sig_mask]

# Plot LISA clusters
fig, ax = plt.subplots(figsize=(10, 10))
borough_ratio.plot(column='quadrant', categorical=True, k=5, cmap='Paired',
                  legend=True, ax=ax)
plt.title('LISA Cluster Map of rule breaking Airbnbs')
plt.axis('off')
plt.savefig('plots/raw/LISA_rule_breaking.png') 
plt.show()

# Additional analysis plots
plt.figure(figsize=(10, 6))
plt.hist(y, bins=20)
plt.title('Distribution of Ratio_of_more_than_90')
plt.xlabel('Value')
plt.show()

print(y.describe())
# print(local_moran.Is.describe())
print(pd.Series(local_moran.Is).describe())

print(f"Number of significant clusters: {(local_moran.p_sim < 0.1).sum()}")

In [ ]:
#| echo: false
#| output: false
# Distance-based weights (20km)
centroids = borough_ratio.geometry.centroid
coords = np.column_stack((centroids.x, centroids.y))
knn = KNN.from_dataframe(borough_ratio, k=4)  # Approximate 20km neighbors
knn.transform = 'r'

# Calculate Local Moran's I with distance weights
local_moran_dist = Moran_Local(y, knn)

# Add results to GeoDataFrame
borough_ratio['Ii_dist'] = local_moran_dist.Is

# Plot results with distance-based weights
fig, ax = plt.subplots(figsize=(12, 8))
borough_ratio.plot(column='Ii_dist', legend=True, ax=ax)
plt.title("Local Moran Statistic (Distance-based)")
plt.axis('off')
plt.show()

In [ ]:
#| echo: false
#| output: false
from libpysal.weights import Queen, lag_spatial
from esda.moran import Moran_BV, Moran_Local_BV

# load data
connect = pd.read_csv("data/connect.csv")
borough = gpd.read_file("data/statistical-gis-boundaries-london/ESRI/London_Borough_Excluding_MHW.shp")

# merge the data
borough_connect = borough.merge(connect, left_on="GSS_CODE", right_on="Borough_code")

In [ ]:
#| echo: false
#| output: false
# analyse the spatial autocorrelation of monthly rent and airbnbs breaking the rule
# Variables
var1 = 'Monthly_rent_2023'
var2 = 'Ratio_of_more_than_90'

# Check for and handle missing data
borough_connect.dropna(subset=[var1, var2], inplace=True)

# Create weights and row-standardize them
weights = Queen.from_dataframe(borough_connect, use_index=True)
weights.transform = 'r'

# Bivariate Moran's I
moran_bv = Moran_BV(borough_connect[var1], borough_connect[var2], weights)
print(f"Bivariate Moran's I between {var1} and {var2}: {moran_bv.I:.3f}")
print(f"p-value: {moran_bv.p_sim:.3f}")

# Bivariate Moran Plot
fig, ax = plt.subplots(figsize=(10, 5))
spatial_lag_var2 = lag_spatial(weights, borough_connect[var2])  # Calculate the spatial lag of var2
scatter = ax.scatter(borough_connect[var1], spatial_lag_var2, color='blue', edgecolor='k', alpha=0.7)
fit = np.polyfit(borough_connect[var1], spatial_lag_var2, 1)
ax.plot(borough_connect[var1], np.polyval(fit, borough_connect[var1]), color='red', linestyle='--', linewidth=1)
ax.set_title('Bivariate Moran Scatter Plot monthly rent and rule breaking Airbnbs')
ax.set_xlabel(var1)
ax.set_ylabel(f"Spatial Lag of {var2}")
plt.savefig('plots/raw/Moran_monthly_rent.png')
plt.show()

# Bivariate Local Moran's I
local_moran_bv = Moran_Local_BV(borough_connect[var1], borough_connect[var2], weights)

# LISA Plot (Bivariate)
fig, ax = plt.subplots(figsize=(10, 10))
borough_connect.assign(cl=local_moran_bv.q).plot(column='cl', categorical=True, 
                                                 cmap='Paired', linewidth=0.1, ax=ax, 
                                                 edgecolor='white', legend=True)
labels = ['Not Significant', 'Low-Low', 'Low-High', 'High-Low', 'High-High']
legend = ax.get_legend()
if legend:
    legend.set_bbox_to_anchor((1, 1))
    legend.set_title('Cluster Type')
    for text, label in zip(legend.get_texts(), labels):
        text.set_text(label)

ax.set_title('Bivariate LISA Cluster Map of monthly rent and rule breaking Airbnbs')
ax.set_axis_off()
plt.savefig('plots/raw/LISA_monthly_rent.png')
plt.show()

In [ ]:
#| echo: false
#| output: false
# analyse the spatial autocorrelation of vacant ratio and airbnbs breaking the rule
# Variables
var1 = 'Vacant_Ratio'
var2 = 'Ratio_of_more_than_90'

# Check for and handle missing data
borough_connect.dropna(subset=[var1, var2], inplace=True)

# Create weights and row-standardize them
weights = Queen.from_dataframe(borough_connect, use_index=True)
weights.transform = 'r'

# Bivariate Moran's I
moran_bv = Moran_BV(borough_connect[var1], borough_connect[var2], weights)
print(f"Bivariate Moran's I between {var1} and {var2}: {moran_bv.I:.3f}")
print(f"p-value: {moran_bv.p_sim:.3f}")

# Bivariate Moran Plot
fig, ax = plt.subplots(figsize=(10, 5))
spatial_lag_var2 = lag_spatial(weights, borough_connect[var2])  # Calculate the spatial lag of var2
scatter = ax.scatter(borough_connect[var1], spatial_lag_var2, color='blue', edgecolor='k', alpha=0.7)
fit = np.polyfit(borough_connect[var1], spatial_lag_var2, 1)
ax.plot(borough_connect[var1], np.polyval(fit, borough_connect[var1]), color='red', linestyle='--', linewidth=1)
ax.set_title('Bivariate Moran Scatter Plot of vacant ratio and rule breaking Airbnbs')
ax.set_xlabel(var1)
ax.set_ylabel(f"Spatial Lag of {var2}")
plt.savefig('plots/raw/Moran_vacant_ratio.png')
plt.show()

# Bivariate Local Moran's I
local_moran_bv = Moran_Local_BV(borough_connect[var1], borough_connect[var2], weights)

# LISA Plot (Bivariate)
fig, ax = plt.subplots(figsize=(10, 10))
borough_connect.assign(cl=local_moran_bv.q).plot(column='cl', categorical=True, 
                                                 cmap='Paired', linewidth=0.1, ax=ax, 
                                                 edgecolor='white', legend=True)
labels = ['Not Significant', 'Low-Low', 'Low-High', 'High-Low', 'High-High']
legend = ax.get_legend()
if legend:
    legend.set_bbox_to_anchor((1, 1))
    legend.set_title('Cluster Type')
    for text, label in zip(legend.get_texts(), labels):
        text.set_text(label)

ax.set_title('Bivariate LISA Cluster Map of vacant ratio and rule breaking Airbnbs')
ax.set_axis_off()
plt.savefig('plots/raw/LISA_vacant_ratio.png')
plt.show()

In [ ]:
#| echo: false
#| output: false
# Plotting the combined figure showing the rusults of Moran scatter plot and LISA cluster map
from PIL import Image, ImageDraw, ImageFont

# Paths to the images
morans = ['plots/raw/Moran_rule_breaking.png', 'plots/raw/Moran_monthly_rent.png', 'plots/raw/Moran_vacant_ratio.png']
lisas = ['plots/raw/LISA_rule_breaking.png', 'plots/raw/LISA_monthly_rent.png', 'plots/raw/LISA_vacant_ratio.png']

# Load all images
images = [Image.open(img) for img in morans + lisas]

# Calculate total width and height for the new image
total_width = images[0].width * 3
max_height = images[0].height + images[3].height 

# Create a new image with the appropriate size
new_im = Image.new('RGB', (total_width, max_height))

# Paste each Moran plot into the new image
for i, img in enumerate(images[:3]):  # First three are Moran plots
    new_im.paste(img, (img.width * i, 0))

# Paste each LISA plot into the new image
for i, img in enumerate(images[3:]):  # Last three are LISA plots
    new_im.paste(img, (img.width * i, images[0].height))  # Paste below the Moran plots

new_im.save('plots/combined_of_Moran_and_LISA.png')

### Figure 1: Results of Moran and LISA analysis of rule breaking Airbnbs, monthly rent and vacancy ratio

![](plots/combined_of_Moran_and_LISA.png)

#### Moran test result
1. Rule-breaking Airbnbs exhibit significant spatial clustering (positive spatial autocorrelation), meaning these properties are not randomly distributed but are concentrated in specific areas, primarily in central and eastern London.
2. The scatter plots further show a positive relationship between rule-breaking Airbnbs and both monthly rents and vacancy ratios, suggesting that areas with higher Airbnb violations also face increased rents and vacancies.

#### LISA test result
The LISA cluster maps highlight “hotspots” where rule-breaking Airbnbs are spatially significant:
1. Rule-breaking Airbnbs: Central London (e.g., Westminster) and parts of eastern London show significant clustering (brown regions).
2. Monthly Rent: High Airbnb violations coincide with high rent clusters (High-High), reinforcing the pressure on housing affordability.
3. Vacancy Rates: Similar High-High clusters are observed in eastern London, suggesting a correlation between high Airbnb violations and increased vacancy rates in certain neighborhoods.
Conversely, areas with Low-Low clusters (e.g., northern and southwestern boroughs) indicate regions with lower Airbnb activity and minimal impact on rents or vacancies.


In [ ]:
#| echo: false
#| output: false
# SAR model
from spreg import ML_Lag
# Import data
data = pd.read_csv("data/connect.csv")
shp = gpd.read_file("data/statistical-gis-boundaries-london/ESRI/London_Borough_Excluding_MHW.shp")

# Merge data and transform coordinate system
zone = shp.merge(data, left_on="GSS_CODE", right_on="Borough_code")
zone = zone.to_crs("EPSG:27700")

# Check and remove missing values
columns = ['Monthly_rent_2023', 'Vacant_Ratio', 'Ratio_of_more_than_90']
print("Missing values:\n", zone[columns].isna().sum())
zone = zone.dropna(subset=columns)

# Construct spatial weights matrix
w = Queen.from_dataframe(zone)
w.transform = 'r'

# Prepare variables
y = zone['Ratio_of_more_than_90'].values.reshape(-1, 1)
X = zone[['Monthly_rent_2023', 'Vacant_Ratio']].values

# Fit Spatial Lag Model
sar_model = ML_Lag(y, X, w=w,
                   name_y='Ratio_of_more_than_90',
                   name_x=['Monthly_rent_2023', 'Vacant_Ratio'],
                   name_w='w')

# Output model results
print("=== SAR Model Results ===")
print(sar_model.summary)

# Visualize residuals
zone['residuals'] = sar_model.u
fig, ax = plt.subplots(figsize=(8, 6))
zone.plot(column='residuals', cmap='viridis', legend=True, ax=ax)
plt.title("SAR Model Residuals")
plt.axis('off')
plt.show()

In [ ]:
#| echo: false
#| output: false
# GWR model
from mgwr.gwr import GWR
from mgwr.sel_bw import Sel_BW
zone = zone.to_crs("EPSG:27700")
zone['centro'] = zone.geometry.centroid
zone['X'] = zone['centro'].x
zone['Y'] = zone['centro'].y
g_y_rent = zone['Monthly_rent_2023'].values.reshape((-1, 1))
g_X_rent = zone[['Ratio_of_more_than_90']].values
g_coords = list(zip(zone['X'], zone['Y']))

# Automatically set bw_min and bw_max based on the number of observations
n_obs = len(g_coords)  # Number of observations
bw_min = 2  # Minimum bandwidth, should be a positive integer
bw_max = max(bw_min, n_obs - 1)  # Ensures bw_max does not exceed n_obs - 1

# Initialize bandwidth selector with dynamic bandwidth settings
gwr_selector_rent = Sel_BW(g_coords, g_y_rent, g_X_rent, fixed=False)

# Search for optimal bandwidth using the golden section search method
gwr_bw_rent = gwr_selector_rent.search(search_method='golden_section', criterion='AICc', bw_min=bw_min, bw_max=bw_max)
print('Optimal Bandwidth Size for Rent:', gwr_bw_rent)

# Fit GWR model with the determined optimal bandwidth
gwr_results_rent = GWR(g_coords, g_y_rent, g_X_rent, gwr_bw_rent, fixed=False, kernel='bisquare').fit()
print(gwr_results_rent.summary())

In [ ]:
#| echo: false
#| output: false
g_coords = list(zip(zone['X'], zone['Y']))

# Define independent and dependent variables for the Vacant_Ratio model
g_y_vacant = zone['Vacant_Ratio'].values.reshape((-1, 1))
g_X_vacant = zone[['Ratio_of_more_than_90']].values

# Automatically set bw_min and bw_max based on the number of observations
n_obs = len(g_coords)  # Number of observations
bw_min = 2  # Minimum bandwidth, should be a positive integer
bw_max = max(bw_min, n_obs - 1)  # Ensures bw_max does not exceed n_obs - 1

# Initialize bandwidth selector with dynamic bandwidth settings for Vacant_Ratio
gwr_selector_vacant = Sel_BW(g_coords, g_y_vacant, g_X_vacant, fixed=False)

# Search for optimal bandwidth using the golden section search method for Vacant_Ratio
gwr_bw_vacant = gwr_selector_vacant.search(search_method='golden_section', criterion='AICc', bw_min=bw_min, bw_max=bw_max)
print('Optimal Bandwidth Size for Vacant Ratio:', gwr_bw_vacant)

# Fit GWR model with the determined optimal bandwidth for Vacant_Ratio
gwr_results_vacant = GWR(g_coords, g_y_vacant, g_X_vacant, gwr_bw_vacant, fixed=False, kernel='bisquare').fit()
print(gwr_results_vacant.summary())

In [ ]:
#| echo: false
#| output: false
zone['coefficient'] = gwr_results_rent.params[:, 1]  # Add coefficients
zone['t_values'] = gwr_results_rent.tvalues[:, 1]  # Add t-values

In [ ]:
#| echo: false
#| output: false
# Define the variable names to be visualized, corresponding to the regression results added
var_names = ['coefficient']  # Adjust this if more variables from the model should be visualized

fig, axes = plt.subplots(1, len(var_names), figsize=(12, 3))

# Ensure `axes` is iterable
if len(var_names) == 1:
    axes = [axes]

for i, var in enumerate(var_names):
    ax = axes[i]  # Access each subplot axis
    zone.plot(column=var, cmap='viridis', legend=True, ax=ax, edgecolor='white', legend_kwds={'label': "Coefficient value"})
    ax.set_title(f'Regression Coefficients for {var}')
    ax.set_axis_off()

    # Highlight non-significant areas based on a significance threshold
    threshold = 1.96
    non_significant = zone['t_values'].abs() < threshold  # Ensuring the use of absolute value for significance checking
    zone.loc[non_significant].plot(ax=ax, color='lightgrey', edgecolor='white')

plt.tight_layout()
plt.show()

In [ ]:
#| echo: false
#| output: false
# Fit GWR for Monthly_rent_2023
gwr_model_rent = GWR(g_coords, zone['Monthly_rent_2023'].values.reshape((-1, 1)),
                     zone[['Ratio_of_more_than_90']].values.reshape((-1, 1)), gwr_bw_rent).fit()

# Fit GWR for Vacant_Ratio
gwr_model_vacant = GWR(g_coords, zone['Vacant_Ratio'].values.reshape((-1, 1)),
                       zone[['Ratio_of_more_than_90']].values.reshape((-1, 1)), gwr_bw_vacant).fit()

# Extract coefficients and t-values for each model
rent_coefs = pd.DataFrame(gwr_model_rent.params, columns=['Intercept', 'Effect_of_Ratio_of_more_than_90_on_Rent'])
rent_tvals = pd.DataFrame(gwr_model_rent.tvalues, columns=['t_Intercept', 't_Effect_on_Rent'])

vacant_coefs = pd.DataFrame(gwr_model_vacant.params, columns=['Intercept', 'Effect_of_Ratio_of_more_than_90_on_Vacancy'])
vacant_tvals = pd.DataFrame(gwr_model_vacant.tvalues, columns=['t_Intercept', 't_Effect_on_Vacancy'])

In [ ]:
#| echo: false
#| output: false
# Add results directly to zone GeoDataFrame
zone['Rent_Effect'] = rent_coefs['Effect_of_Ratio_of_more_than_90_on_Rent']
zone['Vacancy_Effect'] = vacant_coefs['Effect_of_Ratio_of_more_than_90_on_Vacancy']

# Check significance and add to zone
zone['Significant_Rent'] = rent_tvals['t_Effect_on_Rent'].abs() > 1.96
zone['Significant_Vacancy'] = vacant_tvals['t_Effect_on_Vacancy'].abs() > 1.96

In [ ]:
#| echo: false
#| output: false
fig, ax = plt.subplots(1, 2, figsize=(12, 6))

# Plot for Rent
zone.plot(column='Rent_Effect', cmap='viridis', ax=ax[0], legend=True,
          legend_kwds={'label': "Effect on Rent"})
zone[~zone['Significant_Rent']].plot(color='lightgrey', ax=ax[0])
ax[0].set_title('Effect of Ratio_of_more_than_90 on Rent')
ax[0].set_axis_off()

# Plot for Vacancy
zone.plot(column='Vacancy_Effect', cmap='viridis', ax=ax[1], legend=True,
          legend_kwds={'label': "Effect on Vacancy"})
zone[~zone['Significant_Vacancy']].plot(color='lightgrey', ax=ax[1])
ax[1].set_title('Effect of Ratio_of_more_than_90 on Vacancy')
ax[1].set_axis_off()

plt.tight_layout()
plt.show()

In [ ]:
#| echo: false
#| output: false
# combing the plots to a new plot
zone['residuals'] = sar_model.u

# Create a figure with three subplots (one row, three columns)
fig, ax = plt.subplots(1, 3, figsize=(18, 6))  # Adjust the figure size as needed

# Plot for Residuals
zone.plot(column='residuals', cmap='viridis', ax=ax[0], legend=True)
ax[0].set_title('SAR Model Residuals')
ax[0].set_axis_off()

# Plot for Rent Effect
zone.plot(column='Rent_Effect', cmap='viridis', ax=ax[1], legend=True, legend_kwds={'label': "Effect on Rent"})
zone[~zone['Significant_Rent']].plot(color='lightgrey', ax=ax[1])
ax[1].set_title('Effect of rule breaking Airbnbs on rent')
ax[1].set_axis_off()

# Plot for Vacancy Effect
zone.plot(column='Vacancy_Effect', cmap='viridis', ax=ax[2], legend=True, legend_kwds={'label': "Effect on Vacancy"})
zone[~zone['Significant_Vacancy']].plot(color='lightgrey', ax=ax[2])
ax[2].set_title('Effect of rule breaking Airbnbs on vacancy')
ax[2].set_axis_off()

#output
plt.savefig('plots/Results_of_SAR_and_GWR_model.png', dpi=600, bbox_inches='tight')  

# Adjust layout
plt.tight_layout()

### Figure 2: Results of SAR and GWR Analysis of the Effect of Rule-Breaking Airbnbs on Monthly Rent and Vacancy Ratio

![](plots/Results_of_SAR_and_GWR_model.png)

### SAR model result
1. The residuals map indicates that the model captures most spatial variations but leaves some unexplained patterns, especially in central boroughs.
2. Effect on Rent: Boroughs with high Airbnb violations (e.g., central and inner eastern areas) show a strong positive effect on monthly rents, confirming that Airbnb activity drives up rental prices.
3. Effect on Vacancy: Significant positive impacts are observed in specific eastern and southern areas, where Airbnb violations correlate with higher housing vacancy rates.

### GWR model result
1. The spatial variability captured by GWR confirms that the relationship between Airbnb violations, rents, and vacancies is non-stationary. In central London, the effect on rents is pronounced, while in eastern boroughs, the effect on vacancy rates is stronger.
2. This spatial heterogeneity indicates that Airbnb’s impact is location-dependent, with the most severe effects concentrated in areas of high demand.

### Conclusion
These patterns underscore Airbnb's role in gentrification, a process where local residents—especially lower-income households—are displaced due to rising living costs Jain et al. (2021). As short-term rentals become more profitable, landlords are incentivized to convert long-term housing into Airbnbs, reducing housing availability for local tenants and driving up competition for remaining rental units Bosma and van Doorn (2024) . The impacts are particularly severe in central neighborhoods like Westminster and spreading into eastern boroughs, where vacancy rates increase, further destabilizing communities.

Gentrification does not only lead to physical displacement but also causes cultural displacement, as long-standing communities lose affordable housing and essential social ties. Local businesses catering to residents may also suffer as short-term tourism replaces neighborhood-oriented consumption patterns.

In essence, Airbnb rule-breaking accelerates the gentrification process by prioritizing tourism-driven economic gains over the housing needs of local communities, exacerbating social inequalities.

### References (for Q7, remember to move them with other references together later)
Bosma, J. R. & van Doorn, N. (2024) The Gentrification of Airbnb: Closing Rent Gaps Through the Professionalization of Hosting. *Space and culture*. [Online] 27 (1), 31–47.
Jain, S. et al. (2021) Nowcasting Gentrification Using Airbnb Data. *Proceedings of the ACM on human-computer interaction*. [Online] 5 (CSCW1), 1–21.

## Sustainable Authorship Tools

Using the Terminal in Docker, you compile the Quarto report using `quarto render <group_submission_file>.qmd`.

Your QMD file should automatically download your BibTeX and CLS files and any other required files. If this is done right after library loading then the entire report should output successfully.

Written in Markdown and generated from [Quarto](https://quarto.org/). Fonts used: [Spectral](https://fonts.google.com/specimen/Spectral) (mainfont), [Roboto](https://fonts.google.com/specimen/Roboto) (<span style="font-family:Sans-Serif;">sansfont</span>) and [JetBrains Mono](https://fonts.google.com/specimen/JetBrains%20Mono) (`monofont`). 

## References

Crommelin, L., Troy, L., Martin, C., & Pettit, C. (2018). Is Airbnb a sharing economy success or an urban menace? *Cities, 72*, 177–185.  
Greater London Authority. (2023). Guidance on short-term and holiday lets in London. Retrieved from https://www.london.gov.uk  
InsideAirbnb. (2023). Inside Airbnb: Adding data to the debate. Retrieved from http://insideairbnb.com  
Wachsmuth, D., & Weisler, A. (2018). Airbnb and the rent gap: Gentrification through the sharing economy. *Environment and Planning A: Economy and Space, 50*(6), 1147–1170.

---